In [1]:
#Для расчета прейскуранта бриллиантов
# работает после кода 23-10-24_упрОбработка выгрузки 1С УТ ДК использование коррекцию вставок 1-5 род_корр штрихкод.ipynb
## на входе данные - for_script24-10-2023\выгрузка15-10-2023_все_склады_ДК_24-10-2023_17-57.xlsx
#
#описание - C:\Users\специалист ао\mypandas\work\2023\23-09-ХХ Переоценка ДК магазины выст\ReadMe2023-10-18.docx

In [1]:
import datetime
datetime.datetime.now().strftime('%d-%m-%Y_%H-%M')

'10-11-2023_18-49'

# imports

In [821]:
import numpy as np
import pandas as pd
import re
import os
import importlib

In [822]:
import module_prgem.dictemeralds as mprgem

In [823]:
import module_prgem.price_em as price_em

In [824]:
importlib.reload(mprgem)#, dcla_em_sto dcol_em_sto

<module 'module_prgem.dictemeralds' from 'C:\\Users\\diva0\\work\\KDM2023\\gems\\ipynb\\module_prgem\\dictemeralds.py'>

In [887]:
importlib.reload(price_em)

<module 'module_prgem.price_em' from 'C:\\Users\\diva0\\work\\KDM2023\\gems\\ipynb\\module_prgem\\price_em.py'>

In [888]:
price_em.vst2list('5ИзумрудПМ-0,250-НРОКр+3/Г1 3ИзумрудПМ-1,500-НРОГ+3/Г3 14Кр57-0,180-4/4А')

[('5', 'ИзумрудПМ', '0,250', 'НРОКр', '3/Г1'),
 ('3', 'ИзумрудПМ', '1,500', 'НРОГ', '3/Г3')]

In [889]:
price_em.check_vstlist(_)

(1.75, 217.5, '150-120-', ['до 0,09', '0,50-0,74'], 'C ok;Q ok;PCS ok;')

In [891]:
os.listdir('../ЦВДК/')

['df_gems.xlsx',
 'pr_emeralds_173_20-04-2023.xlsx',
 '~$pr_emeralds_173_20-04-2023.xlsx',
 '~$пром_рез_изумр_проверка_по.xlsx',
 'пром_рез_изумр_проверка_по.xlsx',
 'ТестЦВДК.xlsx']

# Загрузка

## Дальше

In [892]:
df_gems = pd.read_excel('../ЦВДК/df_gems.xlsx')

In [890]:
df_gems.columns

NameError: name 'df_gems' is not defined

In [11]:
df_gems.rename(columns={'Вставки: корректировка\n(по умолчанию стоят данные из базового файла 23-01-01 Сводка)'\
                   :'Вставки_д28'}, inplace=True)

In [15]:
df_vst=df_gems[~df_gems['Вставки_д28'].isna()][['Код','Вставки_д28']].copy(deep=True)

In [65]:
df_vst['Вставки_д28'].unique()

array(['Гранат Фианит', 'Хромдиопсид Фианит', 'ТопазГ Фианит', ...,
       'Фианит Цирконий', 'Жемчуг Раухтопаз', 'Аметист Цирконий'],
      dtype=object)

In [66]:
len(_)

21874

## Наличие в ДК

In [68]:
df_15_10_2023 = pd.read_excel('../from_pereozenka/for_script07-11-2023/выгрузка15-10-2023_все_склады_ДК__08-11-2023_14-25_res_pr.xlsx')

In [69]:
df_15_10_2023.columns

Index(['Unnamed: 0', 'Код', 'Старый код (Справочник "Номенклатура" (Общие))',
       'Склад', 'Склад.Склад / Магазин',
       'Вставки (Справочник "Номенклатура" (Общие))', 'Вид номенклатуры',
       'Тип номенклатуры', 'Номенклатура', 'Группа',
       'Заводской номер (Справочник "Номенклатура" (Общие))',
       'Проба (Справочник "Номенклатура" (Общие))',
       'Цвет металла (Справочник "Номенклатура" (Общие))',
       'Дизайн (Справочник "Номенклатура" (Общие))', 'Артикул ',
       'Размерная группа Центрального камня (Справочник "Номенклатура" (Общие))',
       'Цвет Центрального Камня (Справочник "Номенклатура" (Общие))',
       'Чистота Центрального Камня (Справочник "Номенклатура" (Общие))',
       'Размер (Справочник "Номенклатура" (Общие))', 'Наименование для печати',
       'Единица измерения веса',
       'Вес Вставок ПДК (Карат) (Справочник "Номенклатура" (Общие))',
       'Кол-во ДК (Справочник "Номенклатура" (Общие))',
       'Вес Вставок ДК (Карат) (Справочник "Номенкла

## merge актуальный остаток на 15-10-2023

In [70]:
df_15_10_2023['Вставки корректировка'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 34269 entries, 0 to 34268
Series name: Вставки корректировка
Non-Null Count  Dtype 
--------------  ----- 
34269 non-null  object
dtypes: object(1)
memory usage: 267.9+ KB


In [73]:
df_vst_15102023 = pd.merge(left=df_15_10_2023['Код'], right=df_vst,
         how='left')

# Emeralds

In [ ]:
#05-07-2022
patternDtest = r"""(?ix)(?P<N>\d*)[-_\s]*(?P<Form>[A-яA-z.]+[-\s]?\d*)[\s-]+    # штуки огранка
               (?P<Carat>\d\d?[.,]?\d*)[\s-]+                                 # карат
               (?P<ColQ>\d+[.-]?\d?[ /]+(?:7[aAаА]|\d\d?))[\s-]*           # ц/д
               (?P<PO>[aAаАбБвВгГbB]+)                                     # по"""

patternG3 = r'''(?ix)(\d*[-_\s]*[A-яA-z]{3,15}[\s-]+\d[.,]?\d*[\s-]+ \d\d?[ /]+\d\d?
                      | \d*[-_\s]*[A-яA-z]{3,15}[\s-]+\d{0,2}[.,]?\d{0,3}[\s-]+ [\d\w]{0,4}[ /]+[\d\w]{0,4}\s
                      |  \d{0,2}[-_\s]{0,2}[A-яA-z]{3,15}[\s-]?(?:\d{1,2}[.,]+\d{1,3})?)'''

pattern_g =  r"""(?ix)(?P<N>\d*)
               (?P<Gem>[A-яA-z.]+)[-]?# штуки gem
               (?P<Carat>\d\d?[.,]?\d*)[\s-]+                                 # карат
               (?P<Form>[A-яA-z.]+[+]?\d*)[\s-]+    
               (?P<ColQ>\d+[.-]?\d?[ /]+(?:7[aAаА]|\d\d?))"""
               

pattern_g =  r"""(?ix)(?P<N>\d*)
               (?P<Gem>[A-яA-z.]{4,10})[-]?# штуки gem
               (?P<Carat>\d\d?[.,]?\d*)[\s-]+                                 # карат
               (?P<Form>[A-яA-z.]{1,5})[+\s-]+ 
               (?P<ColQ>\d+[ /]+(?:[Гг]|\d\d?))
               """

vst = '2СапфирПТ-1,540-Кв+2/2 18КР57-0,145-6/6А 2КР57-0,016-4/5А'


re.findall(pattern_g, vst )

In [95]:
df_vst_15102023

,Код,Вставки_д28,emeralds
0,ЦБ-00200503,"45КР57-0,520-4/5А",NaN
1,ЦБ-00002854,"2Кр57-0,083-2/2А 4Кр57-0,084-2/2А 6Кр57-0,280-...",NaN
2,ЦБ-00019951,"1Кр57-0,086-4/5А",NaN
3,ЦБ-00019953,"3Кр57-0,06-4/5А",NaN
4,ЦБ-00111074,"1Кр57-0,050-4/5А 1Кр57-0,031-6/6А 1Кр57-0,014-...",NaN
...,...,...,...
34264,ЦБ-00182016,"СапфирВ 6КР57-0,080-3/4А",NaN
34265,ЦБ-00182017,"СапфирВ 6КР57-0,080-3/4А",NaN
34266,ЦБ-00182019,"ИзумрудВ 4КР57-0,040-2/3А",NaN
34267,ЦБ-00182020,"ИзумрудВ 4КР57-0,040-2/3А",NaN


# df

In [321]:
# на входе текст
t1 = '5ИзумрудПМ-0,250-НРОКр+3/Г3 3ИзумрудПМ-1,500-НРОГ+3/Г3 14Кр57-0,180-4/4А'
t2 = '1изумрудпм-0,356-нроов+3/г3 18кр57-0,125-2/5а'

In [331]:
def vst2list(t):
    pattern_em =  r"""(?ix)(?P<N>\d{,3})# штуки 
               (?P<Gem>изумру[A-яA-z.]{1,5})[-]+#gem
               (?P<Carat>\d\d?[.,]?\d*)[\s-]+                                 # карат
               (?P<Form>[A-яA-z.\d-]{1,10})[+]+#[+\s-]+ 
               (?P<ColQ>\d+[ /]+(?:\d*[ГгКкKk]*\d*))"""
    res = re.findall(pattern_em, t )
    return res if len(res)!=0 else np.nan 

## разработка def check_vstlist

In [841]:
from module_prgem.dictemeralds import dcla_em_sto, dcol_em_sto


In [867]:
pr_em_sto.dtypes

ТУ           object
Обработка    object
Огранка      object
цвет         object
чистота      object
size         object
price         int64
dtype: object

In [866]:
sizes_g = pd.read_excel('module_prgem/size_em_sto.xlsx')
pr_em_sto = pd.read_excel('module_prgem/df_pr_emeralds_sto_173_20-04-2023.xlsx', dtype={'цвет':'str'})

In [877]:

sizes_g = pd.read_excel('module_prgem/size_em_sto.xlsx')
pr_em_sto = pd.read_excel('module_prgem/df_pr_emeralds_sto_173_20-04-2023.xlsx', dtype={'цвет':'str'})

def size_em_sto(q,carat_1):
    '''функция для определения размерной группы изумруда по сто в зависимоти кабашон или фацет огранка
   q - качество, carat_1'''   
    if q not in dcla_em_sto.values():
        return 'error_size_em_sto: q not in dcla_em_sto'
    
    if q.__contains__('Г'):
        sizex = sizes_g[sizes_g['Огранка']=='фацетный вид огранки']
    else:
        sizex = sizes_g[sizes_g['Огранка']=='кобашенный вид огранки']
            
    return sizex[(sizex.low_grade<=carat_1)].sort_values('low_grade').iloc[-1,0]
                        
def check_vstlist(l):
    '''функция для определения размерной группы изумруда по сто в зависимотс кабашон или фацет огранка
   q - качество, carat_1'''                           
    #проверка на пусто или []
    if l is np.nan or len(l)==0:
        return np.nan
    #создать внутренний dfx
    dfx = pd.DataFrame({'vstlist':l})
    cols = ["PCS","GEM","CARAT", "FORM", "CQ"]
    for c in range(len(cols)):
        dfx[cols[c]] = dfx.vstlist.apply(lambda x: x[c].lower().strip())
    dfx['C'] = dfx['CQ'].apply(lambda x: x.split('/')[0].lower().strip())
    dfx['Q'] = dfx['CQ'].apply(lambda x: x.split('/')[1].lower().strip())
    
    
    #пустить проверки по словарям
    report = ''
    report += 'C ok;' if dfx['C'].apply(lambda x: x in dcol_em_sto).all() \
                      else 'C error;'
    report+='Q ok;' if dfx['Q'].apply(lambda x: x in dcla_em_sto).all()\
                      else 'Q error;'
    report+='PCS ok;' if dfx['PCS'].apply(lambda x: x!='').all()\
                      else 'PCS error;'
    #масса за камень
    try:
        dfx['PCS'] = dfx['PCS'].astype('int')
        dfx['CARAT'] = dfx['CARAT'].str.replace(',','.').astype('float')
        dfx['PCS'] = dfx['PCS'].astype('int')
        dfx['C'] = dfx['C'].map(dcol_em_sto)
        dfx['Q'] = dfx['Q'].map(dcla_em_sto)
        dfx['CARAT_1'] = dfx['CARAT']/dfx['PCS']
        dfx['size'] = dfx.apply(lambda x:size_em_sto(x['Q'],x['CARAT_1']),axis=1)
        dfx = pd.merge(left=dfx, right=pr_em_sto,
            how='left', 
            left_on=['size', 'C', 'Q',],
            right_on=['size','цвет', 'чистота'])
        dfx['prcost'] = dfx['CARAT']*dfx['price']

    except:
        return 'except; '+report

    
    return dfx['CARAT'].sum(),dfx['prcost'].sum(),dfx['price'].astype('str').sum(),list(dfx['size']),len(pr_em_sto), report

In [ ]:






#to def

dfx['CARAT_1'] = dfx['CARAT']/dfx['PCS']
dfx['size'] = dfx.apply(lambda x:size_em_sto(x['Q'],x['CARAT_1']),axis=1)
pd.merge(left=dfx, right=pr_em_sto,
         how='left', 
         left_on=['size', 'C', 'Q',],
         right_on=['size','цвет', 'чистота'])


In [870]:
t1 = '1ИзумрудПМ-0,250-НРОКр+3/Г3 1ИзумрудПМ-1,500-НРОГ+3/г3 14Кр57-0,180-4/4А'

In [878]:
check_vstlist(vst2list(t1))

(1.75, 357.5, '110220', ['0,25-0,49', '1,50-1,99'], 230, 'C ok;Q ok;PCS ok;')

In [593]:
(tests_errors[1][::-1]+'1')[::-1]

'1ИзумрудПМ-0,280-НРООв+3/Г3 3КР57-0,040-3/3А'

In [595]:
tests_errors[0]

'ИзумрудПМ-7,900-И41+4/Г3 59КР57-0,690-4/4А 43КР57-0,800-3/5А'

In [596]:
check_vstlist(vst2list(_))

'except; C ok;Q ok;PCS error;'

In [591]:
t2+='1'
t2[::-1]

'11111111а5/2-521,0-75рк81 3г/3+воорн-653,0-мпдурмузи1'

In [378]:
#def check_vstlist(l):

l = vst2list(t1)
l is np.nan or len(l)==0

False

In [379]:
dfx = pd.DataFrame({'vstlist':l})

In [380]:
dfx

,vstlist
0,"(5, ИзумрудПМ, 0,250, НРОКр, 3/Г3)"
1,"(3, ИзумрудПМ, 1,500, НРОГ, 3/Г3)"


In [381]:
cols = ["PCS","GEM","CARAT", "FORM", "CQ"]
for c in range(len(cols)):
    dfx[cols[c]] = dfx.vstlist.apply(lambda x: x[c].lower().strip())

In [382]:
dfx['C'] = dfx['CQ'].apply(lambda x: x.split('/')[0].lower().strip())
dfx['Q'] = dfx['CQ'].apply(lambda x: x.split('/')[1].lower().strip())

In [383]:
dfx

,vstlist,PCS,GEM,CARAT,FORM,CQ,C,Q
0,"(5, ИзумрудПМ, 0,250, НРОКр, 3/Г3)",5,изумрудпм,"0,250",нрокр,3/г3,3,г3
1,"(3, ИзумрудПМ, 1,500, НРОГ, 3/Г3)",3,изумрудпм,"1,500",нрог,3/г3,3,г3


In [384]:
dfx['CARAT'] = dfx['CARAT'].str.replace(',','.').astype('float')

In [385]:
dfx['CARAT'].sum()

1.75

In [386]:
# test

In [387]:
vst2list(t1)

[('5', 'ИзумрудПМ', '0,250', 'НРОКр', '3/Г3'),
 ('3', 'ИзумрудПМ', '1,500', 'НРОГ', '3/Г3')]

In [335]:
vst2list('hjkhkljkoj')

nan

In [509]:
df = df_vst_15102023

In [510]:
df['vst2list'] = df['Вставки_д28'].map((vst2list))[~df['Вставки_д28'].map(vst2list).isna()]

In [511]:
df['vst2list'].map(check_vstlist) [~df['vst2list'].map(check_vstlist).isna()]

160      (1.27, C ok;Q error;PCS ok;)
249          (0.1, C ok;Q ok;PCS ok;)
345        (0.234, C ok;Q ok;PCS ok;)
372         (0.34, C ok;Q ok;PCS ok;)
374        (0.159, C ok;Q ok;PCS ok;)
                     ...             
31958       (0.54, C ok;Q ok;PCS ok;)
31971       (0.27, C ok;Q ok;PCS ok;)
31975       (0.67, C ok;Q ok;PCS ok;)
32597      (0.166, C ok;Q ok;PCS ok;)
32933      (0.203, C ok;Q ok;PCS ok;)
Name: vst2list, Length: 304, dtype: object

## разработка массовая 

In [194]:
df = df_vst_15102023

In [205]:
#10-11-2023-20-40
pattern_em =  r"""(?ix)(?P<N>\d{,3})# штуки 
               (?P<Gem>изумру[A-яA-z.]{1,5})[-]+#gem
               (?P<Carat>\d\d?[.,]?\d*)[\s-]+                                 # карат
               (?P<Form>[A-яA-z.\d-]{1,10})[+]+#[+\s-]+ 
               (?P<ColQ>\d+[ /]+(?:\d*[ГгКкKk]*\d*))"""
               

In [212]:
#в разработке с *цвет
pattern_em2 =  r"""(?ix)(?P<N>\d{,3})# штуки 
               (?P<Gem>изумру[A-яA-z.]{1,5})[-]+#gem
               (?P<Carat>\d\d?[.,]?\d*)[\s-]+                                 # карат
               (?P<Form>[A-яA-z.\d-]{1,10})[+]+#[+\s-]+ 
               (?P<ColQ>\d+[ /]+(?:\d*[ГгКкKk]*\d*)[*]*[A-яA-z]*)"""
               

In [167]:
# чисто для ИзумрудП
pattern_jem = r"""(?ix)(?P<Gem>изумрудп)+#gem"""

In [169]:
#проверка
re.findall(pattern_jem, '9ИзумрудПМ-0,15-Пр-21+3/г3*ghgjg 2КР57-0,120-3/2А ИзумрудПМ')

['ИзумрудП', 'ИзумрудП']

In [209]:
re.findall(pattern_em, '9ИзумрудПМ-0,15-jffkj-kh+3/г3*ghgjg 2КР57-0,120-3/2А')

[('9', 'ИзумрудПМ', '0,15', 'jffkj-kh', '3/г3')]

In [213]:
re.findall(pattern_em2,'1изумрудпм-0,760-нрокр-кр+3/г3*зеленый 2кр57-0,050-4/3а 7кр57-0,050-3/4а')

[('1', 'изумрудпм', '0,760', 'нрокр-кр', '3/г3*зеленый')]

### массовая прогонка pattern

In [214]:
df['emerald_list'] = df['Вставки_д28'].apply(lambda x: re.findall(pattern_em, x ))

In [215]:
df['len_jem'] = df['Вставки_д28'].apply(lambda x: len(re.findall(pattern_jem, x )))

In [216]:
df['len_em'] = df['emerald_list'].apply(lambda x: len( x ))

In [217]:
df[df['Вставки_д28'].str.lower().str.contains('изумрудп')].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 160 to 32933
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Код           300 non-null    object
 1   Вставки_д28   300 non-null    object
 2   emeralds      300 non-null    object
 3   re01          300 non-null    object
 4   emerald_list  300 non-null    object
 5   len_jem       300 non-null    int64 
 6   len_em        300 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 18.8+ KB


In [218]:
df[df['len_em']!=df['len_jem'] ]

,Код,Вставки_д28,emeralds,re01,emerald_list,len_jem,len_em
6470,ЦБ-00155200,"1Изумруд-0,096-НРОКр+3/Г3 12Кр17-0,090-2/2А",NaN,"[(1, Изумруд, 0,096, НРОКр, 3/Г3)]","[(1, Изумруд, 0,096, НРОКр, 3/Г3)]",0,1
11847,ЦБ-00039564,"3Изумруд-0,080-НРОКр+3/Г1 1Кр57-0,440-3/2А",NaN,"[(3, Изумруд, 0,080, НРОКр, 3/Г1)]","[(3, Изумруд, 0,080, НРОКр, 3/Г1)]",0,1
17937,ЦБ-00181165,"1Изумруд-0,210-НРОКр+3/Г3 2КР57-0,050-3/3А 1КР...",NaN,"[(1, Изумруд, 0,210, НРОКр, 3/Г3)]","[(1, Изумруд, 0,210, НРОКр, 3/Г3)]",0,1
29167,ЦБ-00051910,"2Изумруд-22,610-НРОФ+5/Г3 148КР57-1,080-3/5А",NaN,"[(2, Изумруд, 22,610, НРОФ, 5/Г3)]","[(2, Изумруд, 22,610, НРОФ, 5/Г3)]",0,1


In [508]:
df['emerald_list'] 

0        []
1        []
2        []
3        []
4        []
         ..
34264    []
34265    []
34266    []
34267    []
34268    []
Name: emerald_list, Length: 34269, dtype: object

### запись в exel массовой прогонки

In [219]:
df.to_excel('test_re01.xlsx')

In [220]:
df.columns

Index(['Код', 'Вставки_д28', 'emeralds', 're01', 'emerald_list', 'len_jem',
       'len_em'],
      dtype='object')

In [221]:
df[['Код', 'Вставки_д28','emerald_list', 'len_em']]

,Код,Вставки_д28,emerald_list
0,ЦБ-00200503,"45КР57-0,520-4/5А",[]
1,ЦБ-00002854,"2Кр57-0,083-2/2А 4Кр57-0,084-2/2А 6Кр57-0,280-...",[]
2,ЦБ-00019951,"1Кр57-0,086-4/5А",[]
3,ЦБ-00019953,"3Кр57-0,06-4/5А",[]
4,ЦБ-00111074,"1Кр57-0,050-4/5А 1Кр57-0,031-6/6А 1Кр57-0,014-...",[]
...,...,...,...
34264,ЦБ-00182016,"СапфирВ 6КР57-0,080-3/4А",[]
34265,ЦБ-00182017,"СапфирВ 6КР57-0,080-3/4А",[]
34266,ЦБ-00182019,"ИзумрудВ 4КР57-0,040-2/3А",[]
34267,ЦБ-00182020,"ИзумрудВ 4КР57-0,040-2/3А",[]


In [223]:
df_exp = df.explode('emerald_list')

In [227]:
df_exp.dropna(subset='emerald_list', inplace=True)

### разбор по  ["PCS","GEM","CARAT", "FORM", "CQ"]

In [393]:
cols = ["PCS","GEM","CARAT", "FORM", "CQ"]
for c in range(len(cols)):
    df_exp[cols[c]] = df_exp.emerald_list.apply(lambda x: x[c].lower().strip())

In [394]:
df_exp

,Код,Вставки_д28,emeralds,re01,emerald_list,len_jem,len_em,PCS,GEM,CARAT,FORM,CQ,C,Q
160,ЦБ-00019785,"1ИзумрудПМ-1,270-НРООв+2/гз 7Кр57-0,100-3/3А","1изумрудпм-1,270-нроов+2/гз 7кр57-0,100-3/3а","[(1, ИзумрудПМ, 1,270, НРООв, 2/г)]","(1, ИзумрудПМ, 1,270, НРООв, 2/г)",1,1,1,изумрудпм,"1,270",нроов,2/г,2,г
249,ЦБ-00019921,"1ИзумрудПМ-0,100-НРОКр+4/Г3 4Кр57-0,040-2/4А","1изумрудпм-0,100-нрокр+4/г3 4кр57-0,040-2/4а","[(1, ИзумрудПМ, 0,100, НРОКр, 4/Г3)]","(1, ИзумрудПМ, 0,100, НРОКр, 4/Г3)",1,1,1,изумрудпм,"0,100",нрокр,4/г3,4,г3
345,ЦБ-00090311,"1ИзумрудПМ-0,234-НРОКр+3/Г2 6Кр57-0,148-2/2А 2...","1изумрудпм-0,234-нрокр+3/г2 6кр57-0,148-2/2а 2...","[(1, ИзумрудПМ, 0,234, НРОКр, 3/Г2)]","(1, ИзумрудПМ, 0,234, НРОКр, 3/Г2)",1,1,1,изумрудпм,"0,234",нрокр,3/г2,3,г2
372,ЦБ-00163308,"1ИзумрудПМ-0,340-НРООв+3/Г3 12Кр17-0,030-2/2А","1изумрудпм-0,340-нроов+3/г3 12кр17-0,030-2/2а","[(1, ИзумрудПМ, 0,340, НРООв, 3/Г3)]","(1, ИзумрудПМ, 0,340, НРООв, 3/Г3)",1,1,1,изумрудпм,"0,340",нроов,3/г3,3,г3
374,ЦБ-00163310,"1ИзумрудПМ-0,159-НРОКр+3/Г2 12Кр57-0,113-3/4А","1изумрудпм-0,159-нрокр+3/г2 12кр57-0,113-3/4а","[(1, ИзумрудПМ, 0,159, НРОКр, 3/Г2)]","(1, ИзумрудПМ, 0,159, НРОКр, 3/Г2)",1,1,1,изумрудпм,"0,159",нрокр,3/г2,3,г2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31958,ЦБ-00171233,"4ИзумрудПМ-0,540-кр+3/Г3 2КР57-0,030-3/4А","4изумрудпм-0,540-кр+3/г3 2кр57-0,030-3/4а","[(4, ИзумрудПМ, 0,540, кр, 3/Г3)]","(4, ИзумрудПМ, 0,540, кр, 3/Г3)",1,1,4,изумрудпм,"0,540",кр,3/г3,3,г3
31971,ЦБ-00171334,"2ИзумрудПМ-0,270-м+2/Г2 20КР57-0,240-3/5А","2изумрудпм-0,270-м+2/г2 20кр57-0,240-3/5а","[(2, ИзумрудПМ, 0,270, м, 2/Г2)]","(2, ИзумрудПМ, 0,270, м, 2/Г2)",1,1,2,изумрудпм,"0,270",м,2/г2,2,г2
31975,ЦБ-00171360,"2ИзумрудПМ-0,670-пр+5/Г1 2КР57-0,030-3/3А","2изумрудпм-0,670-пр+5/г1 2кр57-0,030-3/3а","[(2, ИзумрудПМ, 0,670, пр, 5/Г1)]","(2, ИзумрудПМ, 0,670, пр, 5/Г1)",1,1,2,изумрудпм,"0,670",пр,5/г1,5,г1
32597,ЦБ-00177187,"2ИзумрудПМ-0,166-НРОКр+3/Г3 6Кр57-0,076-4/5А 6...","2изумрудпм-0,166-нрокр+3/г3 6кр57-0,076-4/5а 6...","[(2, ИзумрудПМ, 0,166, НРОКр, 3/Г3)]","(2, ИзумрудПМ, 0,166, НРОКр, 3/Г3)",1,1,2,изумрудпм,"0,166",нрокр,3/г3,3,г3


In [395]:
for c in cols:
    print(c, df_exp[c].unique())

PCS ['1' '3' '2' '' '218' '32' '7' '5' '4' '12' '6' '8' '17' '48' '18' '60']
GEM ['изумрудпм' 'изумруд' 'изумрудпт']
CARAT ['1,270' '0,100' '0,234' '0,340' '0,159' '0,420' '0,118' '1,134' '0,356'
 '0,170' '0,380' '1,490' '0,680' '0,520' '7,900' '0,450' '0,744' '5,230'
 '0,860' '0,160' '1,205' '3,990' '0,275' '0,590' '0,623' '0,640' '0,530'
 '0,900' '0,193' '0,875' '0,260' '0,219' '0,747' '0,090' '0,550' '0,150'
 '0,080' '0,110' '2,370' '0,722' '0,760' '0,650' '0,437' '0,717' '0,300'
 '0,662' '0,136' '0,200' '1,690' '0,836' '1,130' '0,250' '1,500' '1,930'
 '0,400' '1,350' '0,415' '0,276' '1,050' '0,500' '0,060' '0,750' '0,930'
 '0,096' '0,364' '0,663' '0,180' '1,880' '0,030' '0,211' '0,050' '0,540'
 '0,310' '2,360' '0,236' '0,403' '0,230' '1,370' '0,700' '0,330' '1,070'
 '2,300' '1,990' '0,800' '0,220' '0,510' '0,305' '0,026' '0,016' '0,020'
 '0,035' '0,071' '0,115' '0,254' '0,070' '0,130' '0,361' '0,366' '1,253'
 '0,345' '0,910' '0,120' '0,055' '0,088' '0,053' '0,430' '0,210' '9,070'
 

In [396]:
df_exp[df_exp['CQ']=='2/г']

,Код,Вставки_д28,emeralds,re01,emerald_list,len_jem,len_em,PCS,GEM,CARAT,FORM,CQ,C,Q
160,ЦБ-00019785,"1ИзумрудПМ-1,270-НРООв+2/гз 7Кр57-0,100-3/3А","1изумрудпм-1,270-нроов+2/гз 7кр57-0,100-3/3а","[(1, ИзумрудПМ, 1,270, НРООв, 2/г)]","(1, ИзумрудПМ, 1,270, НРООв, 2/г)",1,1,1,изумрудпм,"1,270",нроов,2/г,2,г


In [397]:
df_exp[df_exp['PCS']=='']

,Код,Вставки_д28,emeralds,re01,emerald_list,len_jem,len_em,PCS,GEM,CARAT,FORM,CQ,C,Q
1181,ЦБ-00160609,"ИзумрудПМ-7,900-И41+4/Г3 59КР57-0,690-4/4А 43К...","изумрудпм-7,900-и41+4/г3 59кр57-0,690-4/4а 43к...","[(, ИзумрудПМ, 7,900, И41, 4/Г3)]","(, ИзумрудПМ, 7,900, И41, 4/Г3)",1,1,,изумрудпм,"7,900",и41,4/г3,4,г3
25598,ЦБ-00181874,"ИзумрудПМ-0,280-НРООв+3/Г3 3КР57-0,040-3/3А","изумрудпм-0,280-нроов+3/г3 3кр57-0,040-3/3а","[(, ИзумрудПМ, 0,280, НРООв, 3/Г3)]","(, ИзумрудПМ, 0,280, НРООв, 3/Г3)",1,1,,изумрудпм,"0,280",нроов,3/г3,3,г3


In [312]:
df_exp['C'] = df_exp['CQ'].apply(lambda x: x.split('/')[0].lower().strip())
df_exp['Q'] = df_exp['CQ'].apply(lambda x: x.split('/')[1].lower().strip())

In [313]:
df_exp

,Код,Вставки_д28,emeralds,re01,emerald_list,len_jem,len_em,PCS,GEM,CARAT,FORM,CQ,C,Q
160,ЦБ-00019785,"1ИзумрудПМ-1,270-НРООв+2/гз 7Кр57-0,100-3/3А","1изумрудпм-1,270-нроов+2/гз 7кр57-0,100-3/3а","[(1, ИзумрудПМ, 1,270, НРООв, 2/г)]","(1, ИзумрудПМ, 1,270, НРООв, 2/г)",1,1,1,изумрудпм,"1,270",нроов,2/г,2,г
249,ЦБ-00019921,"1ИзумрудПМ-0,100-НРОКр+4/Г3 4Кр57-0,040-2/4А","1изумрудпм-0,100-нрокр+4/г3 4кр57-0,040-2/4а","[(1, ИзумрудПМ, 0,100, НРОКр, 4/Г3)]","(1, ИзумрудПМ, 0,100, НРОКр, 4/Г3)",1,1,1,изумрудпм,"0,100",нрокр,4/г3,4,г3
345,ЦБ-00090311,"1ИзумрудПМ-0,234-НРОКр+3/Г2 6Кр57-0,148-2/2А 2...","1изумрудпм-0,234-нрокр+3/г2 6кр57-0,148-2/2а 2...","[(1, ИзумрудПМ, 0,234, НРОКр, 3/Г2)]","(1, ИзумрудПМ, 0,234, НРОКр, 3/Г2)",1,1,1,изумрудпм,"0,234",нрокр,3/г2,3,г2
372,ЦБ-00163308,"1ИзумрудПМ-0,340-НРООв+3/Г3 12Кр17-0,030-2/2А","1изумрудпм-0,340-нроов+3/г3 12кр17-0,030-2/2а","[(1, ИзумрудПМ, 0,340, НРООв, 3/Г3)]","(1, ИзумрудПМ, 0,340, НРООв, 3/Г3)",1,1,1,изумрудпм,"0,340",нроов,3/г3,3,г3
374,ЦБ-00163310,"1ИзумрудПМ-0,159-НРОКр+3/Г2 12Кр57-0,113-3/4А","1изумрудпм-0,159-нрокр+3/г2 12кр57-0,113-3/4а","[(1, ИзумрудПМ, 0,159, НРОКр, 3/Г2)]","(1, ИзумрудПМ, 0,159, НРОКр, 3/Г2)",1,1,1,изумрудпм,"0,159",нрокр,3/г2,3,г2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31958,ЦБ-00171233,"4ИзумрудПМ-0,540-кр+3/Г3 2КР57-0,030-3/4А","4изумрудпм-0,540-кр+3/г3 2кр57-0,030-3/4а","[(4, ИзумрудПМ, 0,540, кр, 3/Г3)]","(4, ИзумрудПМ, 0,540, кр, 3/Г3)",1,1,4,изумрудпм,"0,540",кр,3/г3,3,г3
31971,ЦБ-00171334,"2ИзумрудПМ-0,270-м+2/Г2 20КР57-0,240-3/5А","2изумрудпм-0,270-м+2/г2 20кр57-0,240-3/5а","[(2, ИзумрудПМ, 0,270, м, 2/Г2)]","(2, ИзумрудПМ, 0,270, м, 2/Г2)",1,1,2,изумрудпм,"0,270",м,2/г2,2,г2
31975,ЦБ-00171360,"2ИзумрудПМ-0,670-пр+5/Г1 2КР57-0,030-3/3А","2изумрудпм-0,670-пр+5/г1 2кр57-0,030-3/3а","[(2, ИзумрудПМ, 0,670, пр, 5/Г1)]","(2, ИзумрудПМ, 0,670, пр, 5/Г1)",1,1,2,изумрудпм,"0,670",пр,5/г1,5,г1
32597,ЦБ-00177187,"2ИзумрудПМ-0,166-НРОКр+3/Г3 6Кр57-0,076-4/5А 6...","2изумрудпм-0,166-нрокр+3/г3 6кр57-0,076-4/5а 6...","[(2, ИзумрудПМ, 0,166, НРОКр, 3/Г3)]","(2, ИзумрудПМ, 0,166, НРОКр, 3/Г3)",1,1,2,изумрудпм,"0,166",нрокр,3/г3,3,г3


In [469]:
df_exp['C'].apply(lambda x: x in dcol_em_sto).all()

True

In [467]:
dcla_em_sto

{'help': ['К1К2Г1Г2Г3'],
 'г3': 'Г3',
 'г2': 'Г2',
 'г1': 'Г1',
 'к1': 'К1',
 'к2': 'К2',
 'k1': 'К1',
 'k2': 'К2'}

In [ ]:
vstavki_d.Д.apply(lambda x: x in dcla_dia).unique()

## Формирование словарей

In [457]:
from rusprice.dictdiamonds import dcol_dia, dcla_dia, dpo_dia, dogr_dia_discount

цвет ['1' '2' '3' '4' '5']
чистота ['К1' 'К2' 'Г1' 'Г2' 'Г3']

In [655]:
dcol_em_sto = {'help':['1', '2', '3', '4', '5'],
               '1':'1' ,
               '2':'2' , 
               '3':'3' ,
               '4':'4',  
               '5':'5'}

In [656]:
dcla_em_sto = {'help':['К1', 'К2', 'Г1', 'Г2', 'Г3'],
                   'г3':'Г3',
                   'г2':'Г2',
                   'г1':'Г1',
                   'к1':'К1', 
                   'к2':'К2',
                   'k1':'К1' , 
                   'k2':'К2'}
                   #   

In [789]:
#%%writefile module_prgem.dictemeralds.py
#словари для изумрудов СТО по прейскуранту 
#цвет ['1', '2', '3', '4', '5']
#чистота ['К1', 'К2', 'Г1', 'Г2', 'Г3']

dcol_em_sto = {'help':['1', '2', '3', '4', '5'],
               '1':'1' ,
               '2':'2' , 
               '3':'3' ,
               '4':'4',  
               '5':'5'}

dcla_em_sto = {'help':['К1', 'К2', 'Г1', 'Г2', 'Г3'],
                   'г3':'Г3',
                   'г2':'Г2',
                   'г1':'Г1',
                   'к1':'К1', 
                   'к2':'К2',
                   'k1':'К1' , 
                   'k2':'К2'}
                   





Writing module_prgem.dictemeralds.py


In [314]:
df_exp['C'].unique()

array(['2', '4', '3', '5', '1'], dtype=object)

In [315]:
df_exp['Q'].unique()

array(['г', 'г3', 'г2', 'к1', 'г1', 'к3'], dtype=object)

In [319]:
df_exp['PCS'].unique()

array(['1', '3', '2', '', '218', '32', '7', '5', '4', '12', '6', '8',
       '17', '48', '18', '60'], dtype=object)

In [ ]:
df_exp[df_exp['PCS']=='']

In [318]:
df_exp['CARAT'].str.replace(',','.').astype('float')

160      1.270
249      0.100
345      0.234
372      0.340
374      0.159
         ...  
31958    0.540
31971    0.270
31975    0.670
32597    0.166
32933    0.203
Name: CARAT, Length: 309, dtype: float64

### Разработка продолжается средняя масса dfx = df_exp tests_errors= list()

In [552]:
tests_errors= list()

In [528]:
#tests_errors.extend(['qq,','wwwww'])

In [553]:
tests_errors

[]

In [512]:
 dfx = df_exp

In [514]:
dfx.columns

Index(['Код', 'Вставки_д28', 'emeralds', 're01', 'emerald_list', 'len_jem',
       'len_em', 'PCS', 'GEM', 'CARAT', 'FORM', 'CQ', 'C', 'Q'],
      dtype='object')

In [568]:
dfx['CARAT'] = dfx['CARAT'].str.replace(',','.').astype('float')

In [543]:
dfx[dfx['PCS']=='']['Вставки_д28']

1181     ИзумрудПМ-7,900-И41+4/Г3 59КР57-0,690-4/4А 43К...
25598          ИзумрудПМ-0,280-НРООв+3/Г3 3КР57-0,040-3/3А
Name: Вставки_д28, dtype: object

In [544]:
del_ind = _.index

In [554]:
tests_errors.extend(list(dfx.loc[del_ind, 'Вставки_д28']))

In [555]:
tests_errors

['ИзумрудПМ-7,900-И41+4/Г3 59КР57-0,690-4/4А 43КР57-0,800-3/5А',
 'ИзумрудПМ-0,280-НРООв+3/Г3 3КР57-0,040-3/3А']

In [556]:
dfx.drop(index=del_ind, inplace=True)

In [560]:
dfx['PCS'] = dfx['PCS'].astype('int')

In [562]:
 dfx['C'] = dfx['C'].map(dcol_em_sto)#, 'Q'

In [563]:
 dfx['Q'] = dfx['Q'].map(dcla_em_sto)#, 

In [565]:
 dfx['CARAT_1'] = dfx['CARAT']/dfx['PCS']

In [567]:
dfx[[ 'PCS', 'GEM', 'CARAT', 'FORM', 'CQ', 'C', 'Q','CARAT_1']].dtypes

PCS          int32
GEM         object
CARAT      float64
FORM        object
CQ          object
C           object
Q           object
CARAT_1    float64
dtype: object

In [603]:
dfx['CARAT_1']

160      1.270000
249      0.100000
345      0.234000
372      0.340000
374      0.159000
           ...   
31958    0.135000
31971    0.135000
31975    0.335000
32597    0.083000
32933    0.033833
Name: CARAT_1, Length: 307, dtype: float64

In [859]:
pr_em_sto#.to_excel('module_prgem/df_pr_emeralds_sto_173_20-04-2023.xlsx', index=None)

,ТУ,Обработка,Огранка,цвет,чистота,size,price
0,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,1,К1,"до 0,49",120
1,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,1,К2,"до 0,49",75
2,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,2,К1,"до 0,49",75
3,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,2,К2,"до 0,49",45
4,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,3,К1,"до 0,49",55
...,...,...,...,...,...,...,...
225,СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,4,Г2,"4,00-4,99",250
226,СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,4,Г3,"4,00-4,99",170
227,СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,5,Г1,"4,00-4,99",50
228,СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,5,Г2,"4,00-4,99",30


In [ ]:
pr_em_sto

## размерности

In [690]:
sizes_g = pr_em_sto[~pr_em_sto[['size','Огранка']].duplicated()][['size','Огранка']]

In [691]:
sizes_g 

,size,Огранка
39,"до 0,49",кобашенный вид огранки
88,"0,50-0,74",кобашенный вид огранки
137,"0,75-0,99",кобашенный вид огранки
186,"1,00-1,49",кобашенный вид огранки
235,"1,50-1,99",кобашенный вид огранки
284,"2,00-2,99",кобашенный вид огранки
333,"3,00-3,99",кобашенный вид огранки
382,"4,00-4,99",кобашенный вид огранки
39,"до 0,09",фацетный вид огранки
93,"0,10-0,24",фацетный вид огранки


In [692]:
sizes_g['size_list'] = sizes_g['size'].apply(lambda x: x.split('-'))

In [693]:
sizes_g 

,size,Огранка,size_list
39,"до 0,49",кобашенный вид огранки,"[до 0,49]"
88,"0,50-0,74",кобашенный вид огранки,"[0,50, 0,74]"
137,"0,75-0,99",кобашенный вид огранки,"[0,75, 0,99]"
186,"1,00-1,49",кобашенный вид огранки,"[1,00, 1,49]"
235,"1,50-1,99",кобашенный вид огранки,"[1,50, 1,99]"
284,"2,00-2,99",кобашенный вид огранки,"[2,00, 2,99]"
333,"3,00-3,99",кобашенный вид огранки,"[3,00, 3,99]"
382,"4,00-4,99",кобашенный вид огранки,"[4,00, 4,99]"
39,"до 0,09",фацетный вид огранки,"[до 0,09]"
93,"0,10-0,24",фацетный вид огранки,"[0,10, 0,24]"


In [694]:
sizes_g['low_grade'] = sizes_g['size_list'].apply(lambda x: x[0].split()[0] if len(x)==1 else x[0])

In [695]:
#sizes_g['low_grade'] = sizes_g['size_list'].apply(lambda x: 0 if len(x)==1 else x[0])

In [696]:
sizes_g['hi_grade'] = sizes_g['size_list'].apply(lambda x: x[0].split()[1] if len(x)==1 else x[1])

In [697]:
sizes_g[['hi_grade','low_grade']].dtypes#str.replase(',','.')#astype('float')

hi_grade     object
low_grade    object
dtype: object

In [698]:
sizes_g 

,size,Огранка,size_list,low_grade,hi_grade
39,"до 0,49",кобашенный вид огранки,"[до 0,49]",до,"0,49"
88,"0,50-0,74",кобашенный вид огранки,"[0,50, 0,74]","0,50","0,74"
137,"0,75-0,99",кобашенный вид огранки,"[0,75, 0,99]","0,75","0,99"
186,"1,00-1,49",кобашенный вид огранки,"[1,00, 1,49]","1,00","1,49"
235,"1,50-1,99",кобашенный вид огранки,"[1,50, 1,99]","1,50","1,99"
284,"2,00-2,99",кобашенный вид огранки,"[2,00, 2,99]","2,00","2,99"
333,"3,00-3,99",кобашенный вид огранки,"[3,00, 3,99]","3,00","3,99"
382,"4,00-4,99",кобашенный вид огранки,"[4,00, 4,99]","4,00","4,99"
39,"до 0,09",фацетный вид огранки,"[до 0,09]",до,"0,09"
93,"0,10-0,24",фацетный вид огранки,"[0,10, 0,24]","0,10","0,24"


In [699]:
sizes_g['hi_grade'] = sizes_g['hi_grade'].apply(lambda x: x.replace(',','.')).astype('float')

In [700]:
sizes_g['low_grade'] = sizes_g['low_grade'].apply(lambda x: x.replace(',','.') if x!='до' else 0).astype('float')

In [835]:
sizes_g

,size,Огранка,size_list,low_grade,hi_grade
39,"до 0,49",кобашенный вид огранки,"[до 0,49]",0.00,0.49
88,"0,50-0,74",кобашенный вид огранки,"[0,50, 0,74]",0.50,0.74
137,"0,75-0,99",кобашенный вид огранки,"[0,75, 0,99]",0.75,0.99
186,"1,00-1,49",кобашенный вид огранки,"[1,00, 1,49]",1.00,1.49
235,"1,50-1,99",кобашенный вид огранки,"[1,50, 1,99]",1.50,1.99
284,"2,00-2,99",кобашенный вид огранки,"[2,00, 2,99]",2.00,2.99
333,"3,00-3,99",кобашенный вид огранки,"[3,00, 3,99]",3.00,3.99
382,"4,00-4,99",кобашенный вид огранки,"[4,00, 4,99]",4.00,4.99
39,"до 0,09",фацетный вид огранки,"[до 0,09]",0.00,0.09
93,"0,10-0,24",фацетный вид огранки,"[0,10, 0,24]",0.10,0.24


In [702]:
sizes_g.dtypes

size          object
Огранка       object
size_list     object
low_grade    float64
hi_grade     float64
dtype: object

In [836]:
sizes_g.to_excel('module_prgem/size_em_sto.xlsx',index=None)

In [777]:
def size_em_sto(q,carat_1):
    if q not in dcla_em_sto.values():
        return 'error_size_em_sto: q not in dcla_em_sto'
    
    if q.__contains__('Г'):
        sizex = sizes_g[sizes_g['Огранка']=='фацетный вид огранки']
    else:
        sizex = sizes_g[sizes_g['Огранка']=='кобашенный вид огранки']
            
    return sizex[(sizex.low_grade<=carat_1)].sort_values('low_grade').iloc[-1,0]
    
    

In [779]:
dfx['size'] = dfx.apply(lambda x:size_em_sto(x['Q'],x['CARAT_1']),axis=1)

In [784]:
pr_em_sto.columns

Index(['ТУ', 'Обработка', 'Огранка', 'цвет', 'чистота', 'size', 'price'], dtype='object')

In [861]:
dfx.columns

Index(['Код', 'Вставки_д28', 'emeralds', 're01', 'emerald_list', 'len_jem',
       'len_em', 'PCS', 'GEM', 'CARAT', 'FORM', 'CQ', 'C', 'Q', 'CARAT_1',
       'size'],
      dtype='object')

In [862]:
dfx

,Код,Вставки_д28,emeralds,re01,emerald_list,len_jem,len_em,PCS,GEM,CARAT,FORM,CQ,C,Q,CARAT_1,size
160,ЦБ-00019785,"1ИзумрудПМ-1,270-НРООв+2/гз 7Кр57-0,100-3/3А","1изумрудпм-1,270-нроов+2/гз 7кр57-0,100-3/3а","[(1, ИзумрудПМ, 1,270, НРООв, 2/г)]","(1, ИзумрудПМ, 1,270, НРООв, 2/г)",1,1,1,изумрудпм,1.270,нроов,2/г,2,NaN,1.270000,error_size_em_sto: q not in dcla_em_sto
249,ЦБ-00019921,"1ИзумрудПМ-0,100-НРОКр+4/Г3 4Кр57-0,040-2/4А","1изумрудпм-0,100-нрокр+4/г3 4кр57-0,040-2/4а","[(1, ИзумрудПМ, 0,100, НРОКр, 4/Г3)]","(1, ИзумрудПМ, 0,100, НРОКр, 4/Г3)",1,1,1,изумрудпм,0.100,нрокр,4/г3,4,Г3,0.100000,"0,10-0,24"
345,ЦБ-00090311,"1ИзумрудПМ-0,234-НРОКр+3/Г2 6Кр57-0,148-2/2А 2...","1изумрудпм-0,234-нрокр+3/г2 6кр57-0,148-2/2а 2...","[(1, ИзумрудПМ, 0,234, НРОКр, 3/Г2)]","(1, ИзумрудПМ, 0,234, НРОКр, 3/Г2)",1,1,1,изумрудпм,0.234,нрокр,3/г2,3,Г2,0.234000,"0,10-0,24"
372,ЦБ-00163308,"1ИзумрудПМ-0,340-НРООв+3/Г3 12Кр17-0,030-2/2А","1изумрудпм-0,340-нроов+3/г3 12кр17-0,030-2/2а","[(1, ИзумрудПМ, 0,340, НРООв, 3/Г3)]","(1, ИзумрудПМ, 0,340, НРООв, 3/Г3)",1,1,1,изумрудпм,0.340,нроов,3/г3,3,Г3,0.340000,"0,25-0,49"
374,ЦБ-00163310,"1ИзумрудПМ-0,159-НРОКр+3/Г2 12Кр57-0,113-3/4А","1изумрудпм-0,159-нрокр+3/г2 12кр57-0,113-3/4а","[(1, ИзумрудПМ, 0,159, НРОКр, 3/Г2)]","(1, ИзумрудПМ, 0,159, НРОКр, 3/Г2)",1,1,1,изумрудпм,0.159,нрокр,3/г2,3,Г2,0.159000,"0,10-0,24"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31958,ЦБ-00171233,"4ИзумрудПМ-0,540-кр+3/Г3 2КР57-0,030-3/4А","4изумрудпм-0,540-кр+3/г3 2кр57-0,030-3/4а","[(4, ИзумрудПМ, 0,540, кр, 3/Г3)]","(4, ИзумрудПМ, 0,540, кр, 3/Г3)",1,1,4,изумрудпм,0.540,кр,3/г3,3,Г3,0.135000,"0,10-0,24"
31971,ЦБ-00171334,"2ИзумрудПМ-0,270-м+2/Г2 20КР57-0,240-3/5А","2изумрудпм-0,270-м+2/г2 20кр57-0,240-3/5а","[(2, ИзумрудПМ, 0,270, м, 2/Г2)]","(2, ИзумрудПМ, 0,270, м, 2/Г2)",1,1,2,изумрудпм,0.270,м,2/г2,2,Г2,0.135000,"0,10-0,24"
31975,ЦБ-00171360,"2ИзумрудПМ-0,670-пр+5/Г1 2КР57-0,030-3/3А","2изумрудпм-0,670-пр+5/г1 2кр57-0,030-3/3а","[(2, ИзумрудПМ, 0,670, пр, 5/Г1)]","(2, ИзумрудПМ, 0,670, пр, 5/Г1)",1,1,2,изумрудпм,0.670,пр,5/г1,5,Г1,0.335000,"0,25-0,49"
32597,ЦБ-00177187,"2ИзумрудПМ-0,166-НРОКр+3/Г3 6Кр57-0,076-4/5А 6...","2изумрудпм-0,166-нрокр+3/г3 6кр57-0,076-4/5а 6...","[(2, ИзумрудПМ, 0,166, НРОКр, 3/Г3)]","(2, ИзумрудПМ, 0,166, НРОКр, 3/Г3)",1,1,2,изумрудпм,0.166,нрокр,3/г3,3,Г3,0.083000,"до 0,09"


In [863]:
dfx.dtypes

Код              object
Вставки_д28      object
emeralds         object
re01             object
emerald_list     object
len_jem           int64
len_em            int64
PCS               int32
GEM              object
CARAT           float64
FORM             object
CQ               object
C                object
Q                object
CARAT_1         float64
size             object
dtype: object

In [864]:
pr_em_sto.dtypes


ТУ           object
Обработка    object
Огранка      object
цвет          int64
чистота      object
size         object
price         int64
dtype: object

In [869]:
pd.merge(left=dfx, right=pr_em_sto,
         how='left', 
         left_on=['size', 'C', 'Q',],
         right_on=['size','цвет', 'чистота'])

,Код,Вставки_д28,emeralds,re01,emerald_list,len_jem,len_em,PCS,GEM,CARAT,...,C,Q,CARAT_1,size,ТУ,Обработка,Огранка,цвет,чистота,price
0,ЦБ-00019785,"1ИзумрудПМ-1,270-НРООв+2/гз 7Кр57-0,100-3/3А","1изумрудпм-1,270-нроов+2/гз 7кр57-0,100-3/3а","[(1, ИзумрудПМ, 1,270, НРООв, 2/г)]","(1, ИзумрудПМ, 1,270, НРООв, 2/г)",1,1,1,изумрудпм,1.270,...,2,NaN,1.270000,error_size_em_sto: q not in dcla_em_sto,NaN,NaN,NaN,NaN,NaN,NaN
1,ЦБ-00019921,"1ИзумрудПМ-0,100-НРОКр+4/Г3 4Кр57-0,040-2/4А","1изумрудпм-0,100-нрокр+4/г3 4кр57-0,040-2/4а","[(1, ИзумрудПМ, 0,100, НРОКр, 4/Г3)]","(1, ИзумрудПМ, 0,100, НРОКр, 4/Г3)",1,1,1,изумрудпм,0.100,...,4,Г3,0.100000,"0,10-0,24",СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,4,Г3,40.0
2,ЦБ-00090311,"1ИзумрудПМ-0,234-НРОКр+3/Г2 6Кр57-0,148-2/2А 2...","1изумрудпм-0,234-нрокр+3/г2 6кр57-0,148-2/2а 2...","[(1, ИзумрудПМ, 0,234, НРОКр, 3/Г2)]","(1, ИзумрудПМ, 0,234, НРОКр, 3/Г2)",1,1,1,изумрудпм,0.234,...,3,Г2,0.234000,"0,10-0,24",СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,3,Г2,120.0
3,ЦБ-00163308,"1ИзумрудПМ-0,340-НРООв+3/Г3 12Кр17-0,030-2/2А","1изумрудпм-0,340-нроов+3/г3 12кр17-0,030-2/2а","[(1, ИзумрудПМ, 0,340, НРООв, 3/Г3)]","(1, ИзумрудПМ, 0,340, НРООв, 3/Г3)",1,1,1,изумрудпм,0.340,...,3,Г3,0.340000,"0,25-0,49",СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,3,Г3,110.0
4,ЦБ-00163310,"1ИзумрудПМ-0,159-НРОКр+3/Г2 12Кр57-0,113-3/4А","1изумрудпм-0,159-нрокр+3/г2 12кр57-0,113-3/4а","[(1, ИзумрудПМ, 0,159, НРОКр, 3/Г2)]","(1, ИзумрудПМ, 0,159, НРОКр, 3/Г2)",1,1,1,изумрудпм,0.159,...,3,Г2,0.159000,"0,10-0,24",СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,3,Г2,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,ЦБ-00171233,"4ИзумрудПМ-0,540-кр+3/Г3 2КР57-0,030-3/4А","4изумрудпм-0,540-кр+3/г3 2кр57-0,030-3/4а","[(4, ИзумрудПМ, 0,540, кр, 3/Г3)]","(4, ИзумрудПМ, 0,540, кр, 3/Г3)",1,1,4,изумрудпм,0.540,...,3,Г3,0.135000,"0,10-0,24",СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,3,Г3,100.0
303,ЦБ-00171334,"2ИзумрудПМ-0,270-м+2/Г2 20КР57-0,240-3/5А","2изумрудпм-0,270-м+2/г2 20кр57-0,240-3/5а","[(2, ИзумрудПМ, 0,270, м, 2/Г2)]","(2, ИзумрудПМ, 0,270, м, 2/Г2)",1,1,2,изумрудпм,0.270,...,2,Г2,0.135000,"0,10-0,24",СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,2,Г2,200.0
304,ЦБ-00171360,"2ИзумрудПМ-0,670-пр+5/Г1 2КР57-0,030-3/3А","2изумрудпм-0,670-пр+5/г1 2кр57-0,030-3/3а","[(2, ИзумрудПМ, 0,670, пр, 5/Г1)]","(2, ИзумрудПМ, 0,670, пр, 5/Г1)",1,1,2,изумрудпм,0.670,...,5,Г1,0.335000,"0,25-0,49",СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,5,Г1,30.0
305,ЦБ-00177187,"2ИзумрудПМ-0,166-НРОКр+3/Г3 6Кр57-0,076-4/5А 6...","2изумрудпм-0,166-нрокр+3/г3 6кр57-0,076-4/5а 6...","[(2, ИзумрудПМ, 0,166, НРОКр, 3/Г3)]","(2, ИзумрудПМ, 0,166, НРОКр, 3/Г3)",1,1,2,изумрудпм,0.166,...,3,Г3,0.083000,"до 0,09",СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,3,Г3,65.0


In [788]:
_.to_excel('../ЦВДК/пром_рез_изумр_проверка_по.xlsx')

# DEL

# Прейскурант обработка пдф отдельно в модуль

In [ ]:
#фацетный вид огранки

In [439]:
pr_g = pd.read_excel('../ЦВДК/pr_emeralds_173_20-04-2023.xlsx', sheet_name='г', header=2)

In [440]:
pr_g.columns

Index(['ТУ', 'Обработка', 'Огранка', 'цвет', 'чистота', 'до 0,09', '0,10-0,24',
       '0,25-0,49', '0,50-0,74', '0,75-0,99', '1,00-1,49', '1,50-1,99',
       '2,00-2,99', '3,00-3,99', '4,00-4,99'],
      dtype='object')

In [239]:
pd.melt?

In [441]:
pr_g = pd.melt(pr_g, id_vars=['ТУ', 'Обработка', 'Огранка', 'цвет', 'чистота'], value_vars=[ 'до 0,09', '0,10-0,24',
       '0,25-0,49', '0,50-0,74', '0,75-0,99', '1,00-1,49', '1,50-1,99',
       '2,00-2,99', '3,00-3,99', '4,00-4,99'], value_name='price', var_name='size')

In [442]:
pr_em_sto_g = pr_g[pr_g['ТУ']=='СТО']

In [443]:
pr_em_sto_g.columns

Index(['ТУ', 'Обработка', 'Огранка', 'цвет', 'чистота', 'size', 'price'], dtype='object')

In [444]:
for c in pr_em_sto_g.columns:
    print(c, pr_em_sto_g[c].unique())

ТУ ['СТО']
Обработка ['промасливание (O) и заполнение трещин (F)']
Огранка ['фацетный вид огранки']
цвет ['1' '2' '3' '4' '5']
чистота ['Г1' 'Г2' 'Г3']
size ['до 0,09' '0,10-0,24' '0,25-0,49' '0,50-0,74' '0,75-0,99' '1,00-1,49'
 '1,50-1,99' '2,00-2,99' '3,00-3,99' '4,00-4,99']
price [ 350  300  150  240   75   90   65   55   45   30   27   20   12  500
  180  200  110  120  100   40  550  400  450  250  130   70   22   13
  700  600  170  280   80   60  900  580  185  320  140   26 1200  750
  380  220  160   50   18 1700 1050  850  440 2500 1550  420  210 3000
 1800  800 1400  520  260 3700 1000  650  360]


In [445]:
#кобаш вид огранки

In [446]:
pr_k = pd.read_excel('../ЦВДК/pr_emeralds_173_20-04-2023.xlsx', sheet_name='к', header=2)

In [447]:
pr_k.columns

Index(['ТУ', 'Обработка', 'Огранка', 'цвет', 'чистота', 'до 0,49', '0,50-0,74',
       '0,75-0,99', '1,00-1,49', '1,50-1,99', '2,00-2,99', '3,00-3,99',
       '4,00-4,99'],
      dtype='object')

In [448]:
pr_k = pd.melt(pr_k, id_vars=['ТУ', 'Обработка', 'Огранка', 'цвет', 'чистота'], value_vars=[ 'до 0,49', '0,50-0,74',
       '0,75-0,99', '1,00-1,49', '1,50-1,99', '2,00-2,99', '3,00-3,99',
       '4,00-4,99'], value_name='price', var_name='size')

In [449]:
pr_k

,ТУ,Обработка,Огранка,цвет,чистота,size,price
0,ГОСТ,заполнение трещин (F),кобашенный вид огранки,1,1,"до 0,49",220
1,ГОСТ,заполнение трещин (F),кобашенный вид огранки,1,2,"до 0,49",200
2,ГОСТ,заполнение трещин (F),кобашенный вид огранки,1,3,"до 0,49",170
3,ГОСТ,заполнение трещин (F),кобашенный вид огранки,1,4,"до 0,49",140
4,ГОСТ,заполнение трещин (F),кобашенный вид огранки,1,5,"до 0,49",100
...,...,...,...,...,...,...,...
387,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,3,К2,"4,00-4,99",110
388,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,4,К1,"4,00-4,99",90
389,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,4,К2,"4,00-4,99",56
390,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,5,К1,"4,00-4,99",17


In [450]:
pr_em_sto_k = pr_k[pr_k['ТУ']=='СТО']

In [451]:
pr_em_sto_k.columns

Index(['ТУ', 'Обработка', 'Огранка', 'цвет', 'чистота', 'size', 'price'], dtype='object')

In [452]:
for c in pr_em_sto_k.columns:
    print(c, pr_em_sto_k[c].unique())

ТУ ['СТО']
Обработка ['промасливание (O) и заполнение трещин (F)']
Огранка ['кобашенный вид огранки']
цвет ['1' '2' '3' '4' '5']
чистота ['К1' 'К2']
size ['до 0,49' '0,50-0,74' '0,75-0,99' '1,00-1,49' '1,50-1,99' '2,00-2,99'
 '3,00-3,99' '4,00-4,99']
price [120  75  45  55  28  17   9 160  85  60  35 185 100  70  40  34  22 225
 135  80  39 280 150 170  95 390 240 140  62 490 310 320 180  73 600 210
 110  90  56]


In [454]:
# слияние
pr_em_sto = pd.concat([pr_em_sto_k,pr_em_sto_g])

In [456]:
dfxx = pr_em_sto
for c in dfxx.columns:
    print(c, dfxx[c].unique())

ТУ ['СТО']
Обработка ['промасливание (O) и заполнение трещин (F)']
Огранка ['кобашенный вид огранки' 'фацетный вид огранки']
цвет ['1' '2' '3' '4' '5']
чистота ['К1' 'К2' 'Г1' 'Г2' 'Г3']
size ['до 0,49' '0,50-0,74' '0,75-0,99' '1,00-1,49' '1,50-1,99' '2,00-2,99'
 '3,00-3,99' '4,00-4,99' 'до 0,09' '0,10-0,24' '0,25-0,49']
price [ 120   75   45   55   28   17    9  160   85   60   35  185  100   70
   40   34   22  225  135   80   39  280  150  170   95  390  240  140
   62  490  310  320  180   73  600  210  110   90   56  350  300   65
   30   27   20   12  500  200  550  400  450  250  130   13  700  900
  580   26 1200  750  380  220   50   18 1700 1050  850  440 2500 1550
  420 3000 1800  800 1400  520  260 3700 1000  650  360]


In [597]:
pr_em_sto.to_excel('', index=None)

,ТУ,Обработка,Огранка,цвет,чистота,size,price
39,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,1,К1,"до 0,49",120
40,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,1,К2,"до 0,49",75
41,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,2,К1,"до 0,49",75
42,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,2,К2,"до 0,49",45
43,СТО,промасливание (O) и заполнение трещин (F),кобашенный вид огранки,3,К1,"до 0,49",55
...,...,...,...,...,...,...,...
535,СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,4,Г2,"4,00-4,99",250
536,СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,4,Г3,"4,00-4,99",170
537,СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,5,Г1,"4,00-4,99",50
538,СТО,промасливание (O) и заполнение трещин (F),фацетный вид огранки,5,Г2,"4,00-4,99",30
